In [ ]:
TODO: PCA partitions

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc

import pickle 

data_location = "../data/aging_brain/"

young = pickle.load(open(data_location + "aging_brain_young.pickle",mode='rb'))
old = pickle.load(open(data_location + "aging_brain_old.pickle",mode='rb'))

filtered = pickle.load(open(data_location + "aging_brain_filtered.pickle",mode='rb'))

batch_encoding = np.loadtxt(data_location + 'aging_batch_encoding.tsv')
batch_encoding = batch_encoding.astype(dtype=bool)

young_mask = np.zeros(37069,dtype=bool)
old_mask = np.zeros(37069,dtype=bool)

young_mask[:young.shape[0]] = True
old_mask[young.shape[0]:] = True

In [2]:
# sc.pp.neighbors(filtered)
# sc.tl.umap(filtered)
# sc.tl.louvain(filtered)
# sc.pl.umap(filtered,color='louvain')


In [3]:
# We must find all ways of combining 8 mice into 2 partitions of 4. 

# First find the binary representation of all numbers up to 256.

up = np.unpackbits(np.arange(256).astype(dtype='uint8')).reshape((256,8))

# Find out how many digits are 1

sums = np.sum(up,axis=1)

# Then select all representations where only 4 positions are set to 1.

partitions = up[:128][sums[:128] == 4]

partitions.shape

(35, 8)

In [ ]:
# Now we must select features that are reasonably predictable within the dataset

In [ ]:
import sys
# sys.path.append('/localscratch/bbrener1/rusty_forest_v3/src')
sys.path.append('../src')
import tree_reader as tr 
import lumberjack

selection_forest = lumberjack.fit(
    young.X,
    header=filtered.var_names,
    trees=100,
    braids=2,
    ifs=700,
    ofs=700,
    ss=500,
    depth=8,
    leaves=10,
    sfr=0,
    norm='l1',
    reduce_input='true',
    reduce_output='false'
)

In [ ]:
selection_forest.set_cache(True)

In [ ]:
selection_forest.self_prediction = selection_forest.predict(young.X)


In [ ]:
feature_residuals = selection_forest.self_prediction.residuals()


In [ ]:
centered = young.X - np.mean(young.X,axis=0)
null_squared_residual = np.power(centered,2)
feature_null = np.sum(null_squared_residual,axis=0) + 1

forest_squared_residuals = np.power(feature_residuals,2)

forest_feature_error = np.sum(forest_squared_residuals,axis=0) + 1
forest_feature_remaining = forest_feature_error/feature_null


In [ ]:
plt.figure()
plt.hist(forest_feature_remaining.copy(),bins=50)
plt.show()

filtered_feature_mask = forest_feature_remaining < .5

print(np.sum(filtered_feature_mask))

In [5]:
filtered_feature_mask = np.zeros(2000,dtype=bool)

for feature in cv_forest.output_features:
    f_i = list(young.var_names).index(feature)
    filtered_feature_mask[f_i] = True

In [6]:
# Now we must take the predictable features and train cross-validated forests on them

In [7]:
young_filtered = young[:,filtered_feature_mask]
young_filtered.shape

(16028, 351)

In [ ]:
cv_forest = lumberjack.fit(
    young_filtered.X,
    header=young_filtered.var_names,
    trees=100,
    braids=2,
    ifs=150,
    ofs=150,
    ss=500,
    depth=8,
    leaves=10,
    sfr=0,
    norm='l1',
    reduce_input='true',
    reduce_output='false'
)

In [ ]:
cv_forest.set_cache(True)
cv_forest.backup("cv_forest_trimmed_extra")

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc

import sys
# sys.path.append('/localscratch/bbrener1/rusty_forest_v3/src')
sys.path.append('../src')
import tree_reader as tr 
import lumberjack

cv_forest = tr.Forest.load('cv_forest_trimmed_extra')

In [ ]:
cv_forest.reset_split_clusters()
cv_forest.interpret_splits(
    k=10,
    pca=False,
    depth=8,
    metric="cosine",
    mode='additive_mean',
    relatives=True
)

In [ ]:
cv_forest.maximum_spanning_tree(mode='samples')

In [ ]:
cv_forest.tsne_coordinates = filtered.obsm['X_umap'][young_mask]
cv_forest.html_tree_summary(n=10)

In [ ]:
cv_forest.self_prediction = cv_forest.predict(cv_forest.output)
cv_forest.self_prediction.node_sample_encoding()

In [ ]:
cv_forest.self_prediction.prediction_report()

In [ ]:
cv_forest.old_prediction = cv_forest.predict(old.X.T[filtered_feature_mask].T)
cv_forest.old_prediction.prediction_report()


In [17]:

# for i,partition in enumerate(partitions):
#     partition = np.array(partition).astype(dtype=bool)
#     partition_mask = np.any(batch_encoding[young_mask,:8][:,partition],axis=1)
#     sample_indices = np.arange(young_filtered.X.shape[0])[partition_mask]
#     sub_forest = cv_forest.derive_samples(sample_indices)
#     sub_forest.backup(f"sub_forest_{i}")

# for i,partition in enumerate(partitions):
#     partition = np.array(partition).astype(dtype=bool)
#     partition_mask = np.any(batch_encoding[young_mask,:8][:,partition],axis=1)
#     sub_forest = tr.Forest.load(f"sub_forest_{i}")
#     sub_forest.self_prediction = sub_forest.predict(young_filtered.X[partition_mask])
#     sub_forest.self_prediction.node_sample_encoding()
#     sub_forest.self_prediction.node_sample_r2()
#     sub_forest.test_prediction = sub_forest.predict(young_filtered.X[~partition_mask])
#     sub_forest.test_prediction.node_sample_encoding()
#     sub_forest.test_prediction.node_sample_r2()
#     sub_forest.backup(f"sub_forest_{i}_cached")
#     del(sub_forest)


for factor_index in range(1,39):
    print("+++++++++++++++++++")
    print(factor_index)
    print("+++++++++++++++++++")
    for i,partition in enumerate(partitions[:6]):
        partition = np.array(partition).astype(dtype=bool)
        partition_mask = np.any(batch_encoding[young_mask,:8][:,partition],axis=1)
        sub_forest = tr.Forest.load(f"sub_forest_{i}_cached")
        factor = sub_forest.split_clusters[factor_index]
        sub_forest.self_prediction.compare_factor_fvu(sub_forest.test_prediction,factor)
        del(sub_forest)

+++++++++++++++++++
1
+++++++++++++++++++


Exception ignored in: <function Forest.__del__ at 0x7f83d6af8b00>
Traceback (most recent call last):
  File "../src/tree_reader.py", line 101, in __del__
    self.release_pool()
  File "../src/tree_reader.py", line 98, in release_pool
    self.pool_object.terminate()
AttributeError: 'Forest' object has no attribute 'pool_object'


Estimating FVU for Factor 1
MannwhitneyuResult(statistic=161449.0, pvalue=2.9112678470500183e-56)
770/776
770/776
Self FVU: 0.9047704172160563
Other FVU: 0.9263741574861104
Self COD: 0.09522958278394367 (0.01896918431552544, 0.13948800591458188)
Other COD: 0.0736258425138896


Exception ignored in: <function Forest.__del__ at 0x7f83d6af8b00>
Traceback (most recent call last):
  File "../src/tree_reader.py", line 101, in __del__
    self.release_pool()
  File "../src/tree_reader.py", line 98, in release_pool
    self.pool_object.terminate()
AttributeError: 'NoneType' object has no attribute 'terminate'


Estimating FVU for Factor 1
MannwhitneyuResult(statistic=175559.0, pvalue=9.474681555579286e-47)
770/778
770/778
Self FVU: 0.9040204270895712
Other FVU: 0.9184193537477711
Self COD: 0.0959795729104288 (0.02049725361132249, 0.14730975571494606)
Other COD: 0.08158064625222894


...

Estimating FVU for Factor 38
MannwhitneyuResult(statistic=2580.0, pvalue=0.0509455774944075)
70/78
70/78
Self FVU: 0.767236068751483
Other FVU: 0.7324990705622401
Self COD: 0.23276393124851702 (0.1139803110399375, 0.401097525269158)
Other COD: 0.26750092943775994


Exception ignored in: <function Forest.__del__ at 0x7f83d6af8b00>
Traceback (most recent call last):
  File "../src/tree_reader.py", line 101, in __del__
    self.release_pool()
  File "../src/tree_reader.py", line 98, in release_pool
    self.pool_object.terminate()
AttributeError: 'NoneType' object has no attribute 'terminate'


Estimating FVU for Factor 38
MannwhitneyuResult(statistic=2849.0, pvalue=0.2475238634705414)
70/78
70/78
Self FVU: 0.7852148805451753
Other FVU: 0.7864490163607221
Self COD: 0.2147851194548247 (0.10908659998202058, 0.3388837334656331)
Other COD: 0.21355098363927794
Estimating FVU for Factor 38
MannwhitneyuResult(statistic=2389.0, pvalue=0.010368484082367131)
70/78
70/78
Self FVU: 0.8017473221905809
Other FVU: 0.8231183794861324
Self COD: 0.19825267780941913 (0.09023626134639541, 0.2940419256173731)
Other COD: 0.17688162051386758


Exception ignored in: <function Forest.__del__ at 0x7f83d6af8b00>
Traceback (most recent call last):
  File "../src/tree_reader.py", line 101, in __del__
    self.release_pool()
  File "../src/tree_reader.py", line 98, in release_pool
    self.pool_object.terminate()
AttributeError: 'NoneType' object has no attribute 'terminate'
Exception ignored in: <function Forest.__del__ at 0x7f83d6af8b00>
Traceback (most recent call last):
  File "../src/tree_reader.py", line 101, in __del__
    self.release_pool()
  File "../src/tree_reader.py", line 98, in release_pool
    self.pool_object.terminate()
AttributeError: 'NoneType' object has no attribute 'terminate'


Estimating FVU for Factor 38
MannwhitneyuResult(statistic=3035.0, pvalue=0.49080966804466525)
70/78
70/78
Self FVU: 0.777331735552291
Other FVU: 0.7693789208581839
Self COD: 0.22266826444770904 (0.10088305652288027, 0.36013581042642046)
Other COD: 0.23062107914181607


In [ ]:


for factor in cv_forest.split_clusters[1:]:
    
#     cv_forest.cv_self_prediction.compare_factor_means(cv_forest.cv_old_prediction,factor,metric='euclidean',plot=['distance'])
#     cv_forest.cv_self_prediction.compare_factor_residuals(cv_forest.cv_old_prediction,factor)
    print(factor.name())
    sm,om = cv_forest.cv_self_prediction.compare_factor_marginals(cv_forest.cv_old_prediction,factor)


In [ ]:
cv_forest.cv_self_prediction.prediction_report()


In [ ]:
cv_forest.cv_old_prediction.prediction_report()

In [ ]:
cv_forest.trim(.05)

In [ ]:
len(cv_forest.nodes())

In [ ]:
sub_forest.self_prediction = sub_forest.predict(young_filtered.X[4000:6000])

In [ ]:
sub_forest.self_prediction.prediction_report()

In [ ]:
sub_forest.tsne_coordinates = young.obsm['X_umap'][4000:6000]
sub_forest.split_clusters[1].plot_sample_counts()

In [ ]:
cv_forest.split_clusters[1].plot_sample_counts()

In [ ]:
sub_forest.nodes()[1].split_cluster